In [1]:
# %pip install flask
# !python -m venv ./venv/
# !.\\venv\\Scripts\\activate.bat
# !pip install werkzeug

In [2]:
import werkzeug
import pandas as pd
import json
import yaml
import numpy as np
from sqlalchemy import create_engine, and_, select, engine
from flask import jsonify, Flask, render_template, request, json, Response

In [3]:
def type_output_file(o_data, f_type):
    
    out = pd.DataFrame([],[])

    if f_type == 'yaml':
        out = yaml.dump(o_data.to_dict(orient='records'))

    if f_type == 'json':
        out = o_data.to_json(orient="records")

    if f_type == 'csv':
        out = o_data.to_csv(sep='§')

    return Response(out, mimetype = f_type, headers={"Content-disposition":"attachment; filename=filename." + f_type + ""})
    

In [4]:
app = Flask(__name__)
# app.config['DEBUG'] = True
path = "C:\\Users\\victo\\PycharmProjects\\BibtexDataQuality\\output\\output.json"

@app.route('/')
def index():
    return '<h1>Tela inicial</h1><p>Este site é um protótipo de API para pesquisa de artigos cientificos.</p>'

#endpoint para filtragem na pesquisa por coluna
@app.route('/query', methods=['GET']) 
@app.route('/query/<output_format>/', methods=['GET'])
def query(output_format = 'json'):
    
    #abre conexao com o banco
    engine = create_engine('sqlite:///Bd_Publications.db', echo=False)
    filter_arg = request.args.to_dict()
    
    #trata as chaves e valores inseridos pelo usuario e retorna uma string sql
    filter_fact = ''
    for col, value in filter_arg.items():
        lp_count = len(filter_arg)-1
        for p in range(lp_count):
            filter_fact = filter_fact + col + " = '" + value + "'"
            filter_fact = filter_fact + " and "
        query_filter = filter_fact[:-4]
    
    query = "SELECT * FROM tb_Publications where " + query_filter + ";"
    
    #executa a busca e retorna o output no formato solicitado
    df_table = pd.read_sql_query(query, engine)
    return type_output_file(df_table, output_format)

#endpoint para string de busca
@app.route('/search/<search_string>/', methods=['GET'])
@app.route('/search/<search_string>/<output_format>', methods=['GET'])
def search(search_string, output_format = 'json'):
    
    #abre conexao com o banco
    engine = create_engine('sqlite:///Bd_Publications.db', echo=False)
    query = "SELECT * FROM tb_Publications;"
    df_table = pd.read_sql_query(query, engine)

    df_table = df_table[df_table.apply(lambda x: x.astype(str).str.contains(search_string, case=False).any(), axis=1)]
    return type_output_file(df_table, output_format)

In [5]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True, use_reloader=False, )

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.133:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Dec/2022 19:20:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2022 19:20:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2022 19:20:40] "GET /query/yaml?publication_year=2016&publisher=IEEE HTTP/1.1" 308 -
127.0.0.1 - - [07/Dec/2022 19:20:40] "GET /query/yaml/?publication_year=2016&publisher=IEEE HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2022 19:21:18] "GET /search/2016/yaml HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2022 19:24:42] "GET /search/2016 HTTP/1.1" 308 -
127.0.0.1 - - [07/Dec/2022 19:24:42] "GET /search/2016/ HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2022 19:24:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2022 19:52:55] "GET /query/yaml/?publication_year=2016&publisher=IEEE HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2022 20:32:14] "GET /query/yaml/?publication_year=2016&publisher=IEEE HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/20